In [1]:
from datasets import load_dataset
import pandas as pd

In [2]:
def extract_answer_from_text(text):
    if pd.isna(text) or text == "":
        return "Answer not found in the response"
    
    
    patterns = [
        r"\\boxed\{([^}]+)\}",                    
        r"\\boxed\{([^}]*\{[^}]*\}[^}]*)\}",      
        r"\*\*Answer:\*\*\s*\\boxed\{([^}]+)\}", 
        r"\*\*Answer:\*\*\s*(.+)",               
        r"Answer:\s*\\boxed\{([^}]+)\}",        
        r"Answer:\s*(.+)",                      
        r"answer is\s*\\boxed\{([^}]+)\}",     
        r"answer is\s*(.+)",                    
    ]
    
    for pattern in patterns:
        match = re.search(pattern, text, re.IGNORECASE | re.DOTALL)
        if match:
            return match.group(1).strip()
    
    return "Answer not found in the response"

In [6]:
from vllm import LLM, SamplingParams
import re

_model = None
_sampling_params = None

def get_math_solution(math_problem):
    global _model, _sampling_params
    if _model is None:
        _sampling_params = SamplingParams(
            temperature=0.1, 
            top_p=0.9,
            max_tokens=2048,  
            stop=["<|im_end|>"] 
        )
        _model = LLM(model="Qwen/Qwen3-8B")

    system_prompt = (
        "You are an expert math problem solver. "
        "Please reason through the following problem step by step. "
        "After your reasoning, clearly state your final answer on a new line by itself in the format:\n\n"
        "**Answer: [final answer]**\n\n"
        "Ensure the final answer is a number or a simplified mathematical expression."
    )

    formatted_prompt = (
        f"<|im_start|>system\n{system_prompt}<|im_end|>\n"
        f"<|im_start|>user\n{math_problem}<|im_end|>\n"
        f"<|im_start|>assistant\n"
    )


    outputs = _model.generate([formatted_prompt], _sampling_params)
    full_response = outputs[0].outputs[0].text


    answer_match = extract_answer_from_text(full_response)
   

    return {
        "solution": full_response,
        "answer": answer_match
    }


In [26]:
nums = 'Alright, let\'s tackle this problem step by step. The user has asked, "2+2=?". I need to provide a clear and concise answer. \n\nFirst, I\'ll consider the basic arithmetic operation of addition. Addition involves combining two numbers to find their total. In this case, both numbers are 2. So, adding 2 and 2 together should give me the sum.\n\nTo verify, I can think of it in terms of counting. If I have two apples and someone gives me another two apples, how many apples do I have in total? That would be four apples. Therefore, 2 plus 2 equals 4.\n\nAlternatively, I can use the number line method. Starting at 2 and moving two units to the right lands me at 4. This visual representation also confirms that 2 + 2 equals 4.\n\nI don\'t see any complications or alternative interpretations of the question. The problem is straightforward, and the answer is a simple whole number. There\'s no need to consider more complex mathematical concepts or any potential for error in this case.\n\nSo, after considering all these factors, I can confidently say that 2 plus 2 equals 4.\n</think>\n\n**Answer:** 4'

In [33]:
answer_match[0]

'**Answer:** 4'

In [37]:
get_math_solution("1+...+10=?")

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|                                                       | 0/1 [00:00<?, ?it/s, est. spe…

{'solution': "Alright, let's tackle this problem step by step. The user is asking for the sum of numbers from 1 to 10. I need to figure out the best way to approach this.\n\nFirst, I remember that there's a formula for the sum of an arithmetic series. An arithmetic series is a sequence of numbers where each term increases by a constant difference. In this case, the difference between each term is 1, so it's an arithmetic series with a common difference of 1.\n\nThe formula for the sum of the first n terms of an arithmetic series is S = n/2 * (a1 + an), where S is the sum, n is the number of terms, a1 is the first term, and an is the last term. Alternatively, another version of the formula is S = n * (a1 + an)/2, which is essentially the same thing.\n\nSo, applying this formula to the problem at hand, we have n = 10 because we're summing from 1 to 10. The first term a1 is 1, and the last term an is 10. Plugging these values into the formula, we get S = 10/2 * (1 + 10).\n\nCalculating th

In [5]:
df = load_dataset("HuggingFaceH4/MATH-500")["test"].to_pandas()

In [6]:
df.head(2)

,problem,solution,answer,subject,level,unique_id
0,"Convert the point $(0,3)$ in rectangular coord...",We have that $r = \sqrt{0^2 + 3^2} = 3.$ Also...,"\left( 3, \frac{\pi}{2} \right)",Precalculus,2,test/precalculus/807.json
1,Define\n\[p = \sum_{k = 1}^\infty \frac{1}{k^2...,We count the number of times $\frac{1}{n^3}$ a...,p - q,Intermediate Algebra,5,test/intermediate_algebra/1994.json


In [ ]:

results = [get_math_solution(problem) for problem in df['problem'].head(100)]


df['llm_solution'] = [result['solution'] for result in results]
df['llm_answer'] = [result['answer'] for result in results]

In [ ]:
df2 = df.iloc[:100]
df2['llm_solution'] = [result['solution'] for result in results]
df2['llm_answer'] = [result['answer'] for result in results]

In [ ]:
df2.head()

In [ ]:
df2.iloc[0]["llm_solution"]

In [70]:
def update_answer_column(df, answer_col='answer', llm_solution_col='llm_solution'):
    import pandas as pd
    df_updated = df.copy()
    
    for idx, row in df_updated.iterrows():
        current_answer = row[answer_col]
        llm_solution = row[llm_solution_col]
        
        if current_answer == "Answer not found in the response" or pd.isna(current_answer):
            extracted_answer = extract_answer_from_text(llm_solution)
            df_updated.at[idx, answer_col] = extracted_answer
    
    return df_updated

update_answer_column(df2)

,problem,solution,answer,subject,level,unique_id,llm_solution,llm_answer
0,"Convert the point $(0,3)$ in rectangular coord...",We have that $r = \sqrt{0^2 + 3^2} = 3.$ Also...,"\left( 3, \frac{\pi}{2} \right)",Precalculus,2,test/precalculus/807.json,"Okay, so I need to convert the rectangular coo...","\boxed{\left(3,"
1,Define\n\[p = \sum_{k = 1}^\infty \frac{1}{k^2...,We count the number of times $\frac{1}{n^3}$ a...,p - q,Intermediate Algebra,5,test/intermediate_algebra/1994.json,"Okay, so I have this problem where I need to e...",Answer not found in the response.
2,"If $f(x) = \frac{3x-2}{x-2}$, what is the valu...",$f(-2)+f(-1)+f(0)=\frac{3(-2)-2}{-2-2}+\frac{3...,\frac{14}{3},Algebra,3,test/algebra/2584.json,"Okay, so I need to find the value of f(-2) + f...",Answer not found in the response.
3,How many positive whole-number divisors does 1...,First prime factorize $196=2^2\cdot7^2$. The ...,9,Number Theory,3,test/number_theory/572.json,"Okay, so I need to figure out how many positiv...",Answer not found in the response.
4,The results of a cross-country team's training...,Evelyn covered more distance in less time than...,\text{Evelyn},Algebra,2,test/algebra/1349.json,"Okay, so I need to figure out which student ha...",Answer not found in the response.
...,...,...,...,...,...,...,...,...
95,The projection of $\begin{pmatrix} 2 \\ y \\ -...,The projection of $\begin{pmatrix} 2 \\ y \\ -...,-4,Precalculus,2,test/precalculus/34.json,"Okay, so I have this problem here where I need...",Answer not found in the response.
96,Find the real roots of\n\[\frac{( x+ 1)(x - 3)...,Multiplying out each numerator and denominator...,1 \pm \sqrt{19},Intermediate Algebra,5,test/intermediate_algebra/662.json,"Okay, so I have this equation to solve, and I ...",Answer not found in the response.
97,A figure skater is facing north when she begin...,Each full circle is 360 degrees. Dividing 360...,\text{east},Prealgebra,1,test/prealgebra/105.json,"Okay, so I have this problem about a figure sk...",Answer not found in the response.
98,Simplify $(-k + 4) + (-2 + 3k)$.,We have $(-k+4) + (-2+3k) = -k + 4 -2 + 3k = \...,2k+2,Prealgebra,2,test/prealgebra/1924.json,"Okay, so I need to simplify the expression (-k...",2k


In [22]:
df2.head()

,problem,solution,answer,subject,level,unique_id
0,"Convert the point $(0,3)$ in rectangular coord...",We have that $r = \sqrt{0^2 + 3^2} = 3.$ Also...,"\left( 3, \frac{\pi}{2} \right)",Precalculus,2,test/precalculus/807.json
1,Define\n\[p = \sum_{k = 1}^\infty \frac{1}{k^2...,We count the number of times $\frac{1}{n^3}$ a...,p - q,Intermediate Algebra,5,test/intermediate_algebra/1994.json
2,"If $f(x) = \frac{3x-2}{x-2}$, what is the valu...",$f(-2)+f(-1)+f(0)=\frac{3(-2)-2}{-2-2}+\frac{3...,\frac{14}{3},Algebra,3,test/algebra/2584.json
3,How many positive whole-number divisors does 1...,First prime factorize $196=2^2\cdot7^2$. The ...,9,Number Theory,3,test/number_theory/572.json
4,The results of a cross-country team's training...,Evelyn covered more distance in less time than...,\text{Evelyn},Algebra,2,test/algebra/1349.json


In [30]:

df_head = df2.head(100).copy()
df_head['llm_solution'] = [result["solution"] for result in results]
df_head['llm_answer'] = [result["answer"] for result in results]

In [32]:
!rm -rf qwen3_8B_solutions_math100.csv

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [33]:
df_head.to_csv("qwen3_8B_solutions_math100.csv")

In [31]:
df_head.head(2)

,problem,solution,answer,subject,level,unique_id,llm_solution,llm_answer
0,"Convert the point $(0,3)$ in rectangular coord...",We have that $r = \sqrt{0^2 + 3^2} = 3.$ Also...,"\left( 3, \frac{\pi}{2} \right)",Precalculus,2,test/precalculus/807.json,"<think>\nOkay, so I need to convert the rectan...","(3, \frac{\pi}{2})"
1,Define\n\[p = \sum_{k = 1}^\infty \frac{1}{k^2...,We count the number of times $\frac{1}{n^3}$ a...,p - q,Intermediate Algebra,5,test/intermediate_algebra/1994.json,"<think>\nOkay, so I need to find a way to expr...",Answer not found in the response.


In [35]:
def is_answer_correct(llm_answer, expected_answer):
    global _model, _sampling_params
    
    if _model is None:
        _sampling_params = SamplingParams(
            temperature=0.1,
            top_p=0.9,
            max_tokens=128,
            stop=["<|im_end|>"]
        )
        _model = LLM(model="Qwen/Qwen3-32B")
    
    system_prompt = (
        "You are an answer validation expert. Your task is to compare two answers "
        "and determine if they are equivalent (represent the same value/solution).\n\n"
        "Consider:\n"
        "- Different formats (e.g., '5', '5.0', 'five')\n"
        "- Mathematical equivalence (e.g., '1/2' vs '0.5')\n"
        "- Unit conversions if applicable\n"
        "- Numerical precision differences\n\n"
        "Return ONLY 'True' if the answers are equivalent, or 'False' if they are not."
    )
    
    user_prompt = (
        f"Compare these two answers:\n"
        f"LLM Answer: {llm_answer}\n"
        f"Expected Answer: {expected_answer}\n\n"
        "Are they equivalent? Respond with only 'True' or 'False'."
    )
    
    formatted_prompt = (
        f"<|im_start|>system\n{system_prompt}<|im_end|>\n"
        f"<|im_start|>user\n{user_prompt}<|im_end|>\n"
        f"<|im_start|>assistant\n"
    )
    
    try:
        outputs = _model.generate([formatted_prompt], _sampling_params)
        response = outputs[0].outputs[0].text.strip().lower()
        
        if 'true' in response:
            return True
        elif 'false' in response:
            return False
        else:
            return False
            
    except Exception as e:
        print(f"Error {e}")
        return False

In [36]:
df_head['correct'] = df_head.apply(
    lambda row: is_answer_correct(row['llm_answer'], row['answer']), 
    axis=1
)

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%| | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 tok

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%| | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 tok

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

...

Processed prompts:   0%| | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 tok

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%| | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 tok

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%| | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 tok

In [38]:
df_head.head()

,problem,solution,answer,subject,level,unique_id,llm_solution,llm_answer,correct
0,"Convert the point $(0,3)$ in rectangular coord...",We have that $r = \sqrt{0^2 + 3^2} = 3.$ Also...,"\left( 3, \frac{\pi}{2} \right)",Precalculus,2,test/precalculus/807.json,"<think>\nOkay, so I need to convert the rectan...","(3, \frac{\pi}{2})",True
1,Define\n\[p = \sum_{k = 1}^\infty \frac{1}{k^2...,We count the number of times $\frac{1}{n^3}$ a...,p - q,Intermediate Algebra,5,test/intermediate_algebra/1994.json,"<think>\nOkay, so I need to find a way to expr...",Answer not found in the response.,False
2,"If $f(x) = \frac{3x-2}{x-2}$, what is the valu...",$f(-2)+f(-1)+f(0)=\frac{3(-2)-2}{-2-2}+\frac{3...,\frac{14}{3},Algebra,3,test/algebra/2584.json,"<think>\nOkay, so I need to find the value of ...",Answer not found in the response.,False
3,How many positive whole-number divisors does 1...,First prime factorize $196=2^2\cdot7^2$. The ...,9,Number Theory,3,test/number_theory/572.json,"<think>\nOkay, so I need to figure out how man...",9,True
4,The results of a cross-country team's training...,Evelyn covered more distance in less time than...,\text{Evelyn},Algebra,2,test/algebra/1349.json,"<think>\nOkay, so I need to figure out which s...",Answer not found in the response.,False


In [39]:
df_head.to_csv("qwen3_8B_solutions_math100_with_correctness.csv")

In [41]:
def is_solution_correct(llm_solution: str) -> bool:
    global _model, _sampling_params
    
    if _model is None:
        _sampling_params = SamplingParams(
            temperature=0.1,
            top_p=0.9,
            max_tokens=128,
            stop=["<|im_end|>"]
        )
        _model = LLM(model="Qwen/Qwen3-8B")
    
    system_prompt = (
        "You are a mathematics solution validation expert. Your task is to analyze "
        "a mathematical solution and determine if it is correct and complete.\n\n"
        "Consider:\n"
        "- Logical reasoning steps\n"
        "- Mathematical operations and calculations\n"
        "- Correct application of formulas and theorems\n"
        "- Consistency throughout the solution\n"
        "- Proper conclusion and answer format\n\n"
        "Return ONLY 'True' if the solution is mathematically correct and complete, "
        "or 'False' if it contains errors, is incomplete, or doesn't make sense."
    )
    
    user_prompt = (
        f"Analyze this mathematical solution:\n\n"
        f"{llm_solution}\n\n"
        "Is this solution mathematically correct and complete? "
        "Respond with only 'True' or 'False'."
    )
    
    formatted_prompt = (
        f"<|im_start|>system\n{system_prompt}<|im_end|>\n"
        f"<|im_start|>user\n{user_prompt}<|im_end|>\n"
        f"<|im_start|>assistant\n"
    )
    
    try:
        outputs = _model.generate([formatted_prompt], _sampling_params)
        response = outputs[0].outputs[0].text.strip().lower()
        if 'true' in response:
            return True
        elif 'false' in response:
            return False
        else:
            print(f"Ambiguous validation response: '{response}'")
            return False
            
    except Exception as e:
        print(f"Error in solution validation: {e}")
        return False

In [42]:
df_head["is_llm_correct"] = df_head["llm_solution"].apply(is_solution_correct)

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%| | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 tok

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%| | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 tok

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

...

Processed prompts:   0%| | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 tok

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%| | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 tok

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%| | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 tok

In [44]:
df_head.head(10)

,problem,solution,answer,subject,level,unique_id,llm_solution,llm_answer,correct,is_llm_correct
0,"Convert the point $(0,3)$ in rectangular coord...",We have that $r = \sqrt{0^2 + 3^2} = 3.$ Also...,"\left( 3, \frac{\pi}{2} \right)",Precalculus,2,test/precalculus/807.json,"<think>\nOkay, so I need to convert the rectan...","(3, \frac{\pi}{2})",True,True
1,Define\n\[p = \sum_{k = 1}^\infty \frac{1}{k^2...,We count the number of times $\frac{1}{n^3}$ a...,p - q,Intermediate Algebra,5,test/intermediate_algebra/1994.json,"<think>\nOkay, so I need to find a way to expr...",Answer not found in the response.,False,True
2,"If $f(x) = \frac{3x-2}{x-2}$, what is the valu...",$f(-2)+f(-1)+f(0)=\frac{3(-2)-2}{-2-2}+\frac{3...,\frac{14}{3},Algebra,3,test/algebra/2584.json,"<think>\nOkay, so I need to find the value of ...",Answer not found in the response.,False,True
3,How many positive whole-number divisors does 1...,First prime factorize $196=2^2\cdot7^2$. The ...,9,Number Theory,3,test/number_theory/572.json,"<think>\nOkay, so I need to figure out how man...",9,True,True
4,The results of a cross-country team's training...,Evelyn covered more distance in less time than...,\text{Evelyn},Algebra,2,test/algebra/1349.json,"<think>\nOkay, so I need to figure out which s...",Answer not found in the response.,False,True
5,A regular hexagon can be divided into six equi...,The side length of the hexagon is equal to the...,42,Prealgebra,2,test/prealgebra/1622.json,"<think>\nOkay, so I need to find the perimeter...",42,True,True
6,What is the smallest positive perfect cube tha...,The sum of three consecutive integers takes th...,27,Number Theory,3,test/number_theory/515.json,"<think>\nOkay, so I need to find the smallest ...",Answer not found in the response.,False,True
7,"The set of points $(x,y,z)$ that satisfy\n\[2x...","For the first line, let $t = 2x = 3y = -z.$ T...",90^\circ,Precalculus,4,test/precalculus/927.json,"<think>\nOkay, so I need to find the angle bet...",Answer not found in the response.,False,True
8,"What is the distance, in units, between the po...",We use the distance formula: \begin{align*}\n...,3\sqrt{13},Algebra,3,test/algebra/2036.json,"<think>\nOkay, so I need to find the distance ...",Answer not found in the response.,False,True
9,The expression $2\cdot 3 \cdot 4\cdot 5+1$ is ...,"By the associative property of multiplication,...",4,Prealgebra,5,test/prealgebra/1139.json,"<think>\nOkay, so I need to figure out how man...",Answer not found in the response.,False,False


In [46]:
df_head.to_csv("with_llm_decision.csv")